# Forensics - D. Log Analysis 2 - Anomaly Lookup

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0, azure-cli-profile 2.1.4<br>
__OS Supported:__<br>
    -  Linux
__Platforms Supported:__<br>
     Azure Notebooks DSVM, with Forensics Tools installed, please check out [Get Started](Forensics - A. Get Started 1 - Provision DSVM with Azure Forensics Tools.ipynb).<br>
     
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides you basic knowledge to use Log Analytics to lookup anomalies.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Initialize Azure Resource Management Clients
2. Create Snapshot and Generate SAS URL

## 1. Initialize Azure Resource Management Clients

In [ ]:
# User Input and Save to Environmental store
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
import SentinelAzure
from SentinelWidgets import *
from SentinelAnomalyLookup import *

from pandas.io.json import json_normalize
from azure.loganalytics import LogAnalyticsDataClient
from azure.loganalytics.models import QueryBody
import sys
import timeit
import datetime as dt
import pandas as pd
import copy
from IPython.display import HTML

# User Input and Save to Environmental store
# Set to True if you want to reset the environmental values
import os
from SentinelWidgets import *
reset_environment_valiables = False
env_dir = %env
env_list = ['tenant_domain', 'subscription_id', 'resource_group']
envs =  {env_list[0]:'', env_list[1]:'', env_list[2]:''}
envs = WidgetViewHelper.set_env(reset_environment_valiables, env_dir, envs)
tenant_domain = envs[env_list[0]]
subscription_id = envs[env_list[1]]
resource_group = envs[env_list[2]]
# Counting visitors
from SentinelLog import *
logger = LogHelper(reset_environment_valiables, env_dir, envs, 'AnomalyLookup')
logger.count_page_view()

In [ ]:
# Authentication to Log Analytics
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant $tenant_domain
la_client = get_client_from_cli_profile(LogAnalyticsManagementClient, subscription_id = subscription_id)
la = SentinelAzure.azure_loganalytics_helper.LogAnalyticsHelper(la_client)
creds, _ = get_azure_cli_credentials(resource="https://api.loganalytics.io")
la_data_client = LogAnalyticsDataClient(creds)

## 2. Create Snapshot and Generate SAS URL

In [ ]:
# Select a workspace
selected_workspace = WidgetViewHelper.select_log_analytics_workspace(la)
display(selected_workspace)

In [ ]:
workspace_id = la.get_workspace_id(selected_workspace.value)
#DateTime format: 2019-05-15T07:05:20.000
qTimestamp = input('DateTime: ')
#Entity format: computer
qEntity = input('Entity for search: ')

In [ ]:
anomaly_lookup = AnomalyLookup(workspace_id, la_data_client)
selected_tables = WidgetViewHelper.select_multiple_tables(anomaly_lookup)
display(selected_tables)

In [ ]:
# This action may take a few minutes or more, please be patient.
start = timeit.default_timer()
anomalies, queries = anomaly_lookup.run(qTimestamp, qEntity, list(selected_tables.value))
display(anomalies)

if queries is not None:
    url = WidgetViewHelper.construct_url_for_log_analytics_logs(tenant_domain, subscription_id, resource_group, selected_workspace.value)
    WidgetViewHelper.display_html(WidgetViewHelper.copy_to_clipboard(url, queries, 'Add queries to clipboard and go to Log Analytics'))

print('==================')
print('Elapsed time: ', timeit.default_timer() - start, ' seconds')

## Feedback

In [ ]:
#1. Do you think this notebook helpful?
logger.ask_is_helpful()

In [ ]:
#2. Is this entity compromised?
anomaly_lookup.ask_is_entity_compromised()

In [ ]:
#3. Appreciate your feedback
logger.ask_feedback()